In [37]:
import difflib
import pysrt
import os,subprocess
import shutil
path = os.getcwd()
def update_srt(filename,x):
#     if os.path.exists(os.path.dirname(path+"/static/static/videos_text/"+filename.split('.txt')[0]+"/")):
#         shutil.rmtree(path+"/static/static/videos_text/"+filename.split('.txt')[0]+"/")
#         os.makedirs(os.path.dirname(path+"/static/static/videos_text/"+filename.split('.txt')[0]+"/"))
#     else:
#         os.makedirs(os.path.dirname(path+"/static/static/videos_text/"+filename.split('.txt')[0]+"/"))

    x=x.replace("  "," ").replace('  ',' ').replace('\r','').replace('\\nn','').replace('\\n','').replace('\\','').replace('\n','').replace('&nbsp;','')
    d = difflib.Differ()
    srt_file=filename.split('.txt')[0]+'.srt'
    subs = pysrt.open(srt_file, encoding='iso-8859-1')
    f= open(filename,"r+")
    y=str(f.readlines())
    y = y.replace("  "," ").replace('  ',' ').replace('\r','').replace('\\nn','').replace('\\n','').replace('\\','').replace('\n','').replace('&nbsp;','')
    y=y[2:len(y)-2].lower()
    x=x.lower()
    y=y.split(' ')
    t=x
    x = x.split(' ')
    i=0
    omit={}
    add ={}
    result1 = list(d.compare(y, x))
    result = list(d.compare(x, y))
    result1 =  [v for v in result1 if v[0] != '?']
    result =  [v for v in result if v[0] != '?']

    omit_word=[]
    add_word=[]
    for l in range(0,len(result1)):
        print i,result1[l]
        if result1[l][0] == '-' and len(result1[l])>2:
            omit['line_no'] = i
            omit['word'] = result1[l][2:len(result1[l])]
            if l+1<len(result1):
                if result1[l+1][0] == '+' or result1[l-1][0] == '+' and l>0:
                    omit['flag'] = 1
                    if result1[l-1][0] == '+' and result1[l+1][0] == '-' :
                        i=i+1
                if result1[l+1][0] == '+' and result1[l-1][0] == '-' and l>0:
                    i=i-1
                else:
                    omit['flag'] = 0 
            omit_word.append(omit.copy())
        if result1[l][0] == '+' and len(result1[l])>2:
            add['line_no'] = i
            add['word'] = result1[l][2:len(result1[l])]
            add['flag'] = 0
            add_word.append(add.copy())
        if l+1<len(result1):
            if result1[l][0] == '-' and result1[l+1][0] != '-':
                i=i-1
        i=i+1
    for l in range(0,len(omit_word)):
        for k in range(0,len(add_word)):
            if add_word[k]['line_no'] == omit_word[l]['line_no']:
                omit_word[l]['flag'] = 1
                add_word[k]['flag'] = 1

    print omit_word
   # print "###############"
    print add_word
    cnt=0
    pl_s=0
    pl_e=0
    for j in range(0,len(subs)):
        txt = subs[j].text.lower().replace("  "," ").replace('  ',' ').replace('\n',"").replace('\r','').replace('&nbsp;','')
        q=txt.split(' ')
        pl_s=pl_e
        pl_e = pl_e+len(q)
        print (txt,"---------------------->",pl_s,pl_e)


        for z in range(0,len(add_word)):
            if add_word[z]['flag'] == 0 and pl_s<=add_word[z]['line_no']<pl_e:
                pos = add_word[z]['line_no'] - pl_s
                q = txt.split(' ')
                q.insert(pos,add_word[z]['word'])
                txt = " ".join(q)
                subs[j].text = txt.replace("  "," ").replace('  ',' ').replace('\n',"").replace('\r','')
                pl_e = pl_e+1
        for i in range(0,len(omit_word)):
            if pl_s<=omit_word[i]['line_no'] <pl_e:
                if omit_word[i]['flag'] == 1:
                    #print omit_word[i]['word']
                    for z in range(0,len(add_word)):
                        if add_word[z]['line_no'] == omit_word[i]['line_no']:
                            #print omit_word[i]
                            if q:
                                try:
                                  #  print txt,pl_s,pl_e,omit_word[i]['word'],omit_word[i]['line_no']
                                    #q.remove(omit_word[i]['word'])
                                    pos = add_word[z]['line_no'] - pl_s
                                    #print pos,"######%%%%%%%%%%%%"
                                    q.pop(pos)
                                    q.insert(pos,add_word[z]['word'])
                                    txt = " ".join(q)
                                    txt=txt.replace("  "," ").replace('  ',' ').replace('\n',"").replace('\r','').replace('&nbsp;','')
                                    subs[j].text = txt
                                    add_word[z]['flag'] = 2
                                    omit_word[i]['flag'] = 2
                                    #print txt
                                    #print omit_word[i]['word'],add_word[z]['word'],omit_word[i]['line_no'],add_word[z]['line_no']
                                except:
                                    #txt=txt.replace(omit_word[i]['word'],add_word[z]['word'])
                                    pass
                               # print txt,pl_s,pl_e,omit_word[i]['word'],omit_word[i]['line_no'],omit_word[i]['flag']

                                

                elif omit_word[i]['flag'] == 0:
                   # print q
                    try:
                        if not q:
                            txt = ""
                        else:
                            q.remove(omit_word[i]['word'])
                            txt = " ".join(q)
                            pl_e = pl_e-1
                            omit_word[i]['flag'] = 2
                    except:
                        pass
                    subs[j].text = txt.replace("  "," ").replace('  ',' ').replace('\n',"").replace('\r','').replace('&nbsp;','')

        
        print (subs[j].text,"---------------------->",pl_s,pl_e)
        pl_s=pl_e
    subs.save(srt_file, encoding='iso-8859-1')
#    subs.save(path+"/static/static/videos_text/""+filename.split('.txt')[0]+"/"+srt_file, encoding='iso-8859-1')
    try:
        os.remove(srt_file.split('.')[0].strip()+".vtt")
        print (filename.split('.txt')[0]+"/"+srt_file.split('.')[0].strip()+".vtt")
        os.remove(filename.split('.txt')[0]+"/"+srt_file.split('.')[0].strip()+".vtt")
    except:
        pass
    final = open(filename,'w+')
    print>>final,t
    final.close()
    command = "ffmpeg -i "+  srt_file+" "+path+srt_file.split('.')[0].strip()+".vtt"
    subprocess.call(command, shell=True)
    command = "ffmpeg -i "+  srt_file+" "+filename.split('.txt')[0]+"/"+srt_file.split('.')[0].strip()+str(flag)+".vtt"
    subprocess.call(command, shell=True)


In [39]:
x="on behalf of the vivekananda study circle the extramural the centre for innovation and indo-german centre for sustainability and i extend a hearty welcome the faculty and students in this lecture by one of the greatest of this era and inspiring bharat ratna dr apj abdul kalam which guys are suspected we should begin today's function the blessings of lord almighty me letters of drawing for the prayer tower samayal samayal samayal mridangam survey bhavantu sukhinah sarve santu niramaya survey bhadrani pashyantu market manaquest the head boy and head girl of 11 sridhar babu and kasturi to present a bouquet to doctor abdul kalam banno request dubbed ghulam 2 august floral tribute to swami vivekananda i now call upon mr raghavan of the extra mural lectures theme to introduce doctor apj abdul kalam to the gather dr apj abdul kalam san 2000 to 2017 of vision find me the lost generation of popular call the people's president 1990 indian institute of space science and technology thiruvananthapuram unacademy dr kalam contribution to public service apartment eminent scientist engineer visionary and humanitarian station to be honoured with the americans top engineering private highest civilian award 90 hours welcome to facebook life of pi ignited mind and india 20-20 have inspired million version on a not enough hindi conclusion introduction and request to begin tutorial lecture on creativity thank you friends good evening to all of you first of all i would like to say that i am very sorry that i got delayed because i got you dab young i could not come office today so i am sorry will compensate subway chemistry professor anand a member of the faculty and year everyone of you migrating and it creates a beautiful day of thinking about swami vivekananda very happy to young people i am delighted to be here at the technology madras and address the members of the renowned place of learning and other present you my friend extra very close my heart and many more because my professor who taught me at design of aircraft he became the director of the light image on a professional antiquity guru 56 class 7 32 1013 madrasi stuff technology have you come have you have you used french fries in my address i would like to share that with the youth present i have ahmed sofa 11 million views 11 million you like you in a decade style various parts of the country i have met and what i do i have been there and in php what to be there expression android truth spy all the text make me to learn something about which i would like to share with you tonight i learnt hair pulling get me what i'm saying what you learn from the use of 11 million people i learn what should be unique that is you you want to be unique that is you but the world around nice to meet you now now question whether you you you you want me you not everybody else intercourse human adequate but not satisfying in the long wishes picture like demo my dear friend that your pic. the hardest battle your fight the hardest battle magic mike and never stop fighting until you arrive at your place xxxii you get jetpack joyride big backup the betterment you don't take that the battle mean you have job for unity warning you must have that battle i need you to check the gold the second one is quite the knowledge factory android 18 the artwork with the devotion and policeman vicky sarthak to your brother for the back to the video to be part that you have more definitely you will become"
update_srt('watchvneKaXXXKtHE_in.txt',x)

0 - 
1 - 
1   on
2   behalf
3   of
4   the
5   vivekananda
6   study
7   circle
8   the
9   extramural
10   the
11   centre
12   for
13   innovation
14   and
15   indo-german
16   centre
17   for
18   sustainability
19   and
20   i
21   extend
22   a
23   hearty
24   welcome
25   the
26   faculty
27   and
28   students
29   in
30   this
31   lecture
32   by
33   one
34   of
35   the
36   greatest
37   of
38   this
39   era
40   and
41   inspiring
42   bharat
43   ratna
44   dr
45   apj
46   abdul
47   kalam
48   which
49   guys
50   are
51   suspected
52   we
53   should
54   begin
55   today's
56   function
57   the
58   blessings
59   of
60   lord
61   almighty
62   me
63   letters
64   of
65   drawing
66   for
67   the
68   prayer
69   tower
70   samayal
71   samayal
72   samayal
73   mridangam
74   survey
75   bhavantu
76   sukhinah
77   sarve
78   santu
79   niramaya
80   survey
81   bhadrani
82   pashyantu
83   market
84   manaquest
85   the
86   head
87   boy
88   and
89   head


NameError: global name 'flag' is not defined